<a href="https://colab.research.google.com/github/NinadShegokar/Langchain/blob/main/SummaryGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain scikit-learn langchain_community tiktoken langchain-openai langchainhub chromadb  sentence-transformers   llama-cpp-python
!pip install pypdf dspy-ai fpdf
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install -qU llama-cpp-python

from IPython.display import clear_output
clear_output()

In [ ]:
#imports
import langchain
from torch import cuda, bfloat16
from fpdf import FPDF
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader,PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain,ConversationalRetrievalChain,StuffDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import CTransformers
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain

In [ ]:
import locale
import dspy
def getpreferredencoding(do_setlocale = True):
  return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!wget "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_K_M.gguf"

--2024-04-06 05:22:19--  https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.103, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/fe/17/fe17596731f84a0d03bece77489780bc7e068323c0aeca88b6393d3e9e65dd49/503580dce392c6e64669ad21a77023ba2a17baa0c381250fb67c11ba6406a85e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27zephyr-7b-beta.Q4_K_M.gguf%3B+filename%3D%22zephyr-7b-beta.Q4_K_M.gguf%22%3B&Expires=1712640139&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMjY0MDEzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ZlLzE3L2ZlMTc1OTY3MzFmODRhMGQwM2JlY2U3NzQ4OTc4MGJjN2UwNjgzMjNjMGFlY2E4OGI2MzkzZDNlOWU2NWRkNDkvNTAzNTgwZGNlMzkyYzZlNjQ2NjlhZDIxYTc3MDIzYm

In [ ]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
# #
# n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
# n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
#
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
#
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/content/zephyr-7b-beta.Q4_K_M.gguf",
    temperature=0.75,
    max_tokens=1000,
    top_p=1,
    n_ctx=35000,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
#
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

embd = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME
)



llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/zephyr-7b-beta.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.at

In [ ]:
#load the pdf files from the path
loader = PyPDFLoader(r'/content/Schizophrenia.pdf')
docs = loader.load()


In [ ]:
def summary_generation(file_path):
    # Map
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    map_template = """The following is a set of documents
    {docs}
    Based on this list of docs, please identify the main themes and concepts
    Prepare a concise summary in the form of bullet points
    Helpful Answer:"""
    map_prompt = PromptTemplate.from_template(map_template)
    map_chain = LLMChain(llm=llm, prompt=map_prompt)
    # Reduce
    reduce_template = """The following is set of summaries:
    {docs}
    Based on this list of docs, please identify the main themes and concepts
    Prepare a concise summary in the form of bullet points.
    Helpful Answer:"""
    reduce_prompt = PromptTemplate.from_template(reduce_template)
        # Run chain
    reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

    # Takes a list of documents, combines them into a single string, and passes this to an LLMChain
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_chain, document_variable_name="docs"
    )

    # Combines and iteravely reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `StuffDocumentsChain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=4000,
    )
        # Combining documents by mapping a chain over them, then combining results
    map_reduce_chain = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="docs",
        # Return the results of the map steps in the output
        return_intermediate_steps=False,
    )

    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=100
    )
    split_docs = text_splitter.split_documents(docs)
    all_summaries=map_reduce_chain.run(split_docs)
    print(all_summaries)


In [ ]:
all=summary_generation(r'/content/Schizophrenia.pdf')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



    - Definition and symptoms of schizophrenia, including positive symptoms (delusions, hallucinations, disorganized thinking, behavioral changes), negative symptoms (lack of emotion, social withdrawal), and cognitive impairments.
    - Prevalence and risk factors, such as male gender and earlier onset in males compared to females, genetic and environmental influences (including childhood adversity, cannabis use during adolescence, infectious diseases, parental age, prenatal nutrition).
    - Outcomes for individuals diagnosed with schizophrenia, including the percentage of people who experience significant improvement versus long-term impairment. Risks associated with schizophrenia include suicide rates and decreased life expectancy due to physical health problems.
    - Treatment options, such as antipsychotic medication, counseling, job training, social rehabilitation, and clozapine for treatment-resistant cases. Involuntary hospitalization may be necessary in situations where ther


llama_print_timings:        load time =    3371.63 ms
llama_print_timings:      sample time =     187.13 ms /   327 runs   (    0.57 ms per token,  1747.42 tokens per second)
llama_print_timings: prompt eval time =  423977.31 ms /   952 tokens (  445.35 ms per token,     2.25 tokens per second)
llama_print_timings:        eval time =  210111.11 ms /   326 runs   (  644.51 ms per token,     1.55 tokens per second)
llama_print_timings:       total time =  636404.46 ms /  1278 tokens
Llama.generate: prefix-match hit



    - Magnitude and impact of schizophrenia: 
      - Approx. 24 million people affected worldwide (1 in 300)
      - Onset during late adolescence and twenties, earlier in men than women
      - Significant distress and impairment in personal, family, social, educational, occupational, and other important areas of life
      - 2-3 times more likely to die early than the general population (often due to physical illnesses)
    - Causes of schizophrenia:
      - Interaction between genes and environmental factors
      - Psychosocial factors may affect onset and course
      - Heavy use of cannabis associated with elevated risk
    - Services for people with schizophrenia:
      - Majority not receiving mental health care (50% in mental hospitals)
      - 31.3% receive specialist mental health care
      - Mental hospitals violate basic human rights of people with schizophrenia and are not effective
      - Efforts to transfer care to community needed, including integration in primary 


llama_print_timings:        load time =    3371.63 ms
llama_print_timings:      sample time =     221.59 ms /   401 runs   (    0.55 ms per token,  1809.65 tokens per second)
llama_print_timings: prompt eval time =  358483.48 ms /   807 tokens (  444.22 ms per token,     2.25 tokens per second)
llama_print_timings:        eval time =  257249.77 ms /   400 runs   (  643.12 ms per token,     1.55 tokens per second)
llama_print_timings:       total time =  618543.68 ms /  1207 tokens
Llama.generate: prefix-match hit



    
- Definition and symptoms:
  - Schizophrenia is a mental disorder characterized by positive symptoms (delusions, hallucinations, disorganized thinking, behavioral changes), negative symptoms (lack of emotion, social withdrawal), and cognitive impairments.
  - Prevalence: approximately 24 million people worldwide are affected (1 in 300).
  - Risk factors: male gender, earlier onset in males compared to females, genetic and environmental influences including childhood adversity, cannabis use during adolescence, infectious diseases, parental age, prenatal nutrition.
- Outcomes:
  - Significant distress and impairment in various areas of life for individuals diagnosed with schizophrenia.
  - Suicide rates are higher among people with schizophrenia compared to the general population.
  - Decreased life expectancy due to physical health problems is also common among people with schizophrenia.
- Treatment:
  - Antipsychotic medication, counseling, job training, social rehabilitation, an


llama_print_timings:        load time =    3371.63 ms
llama_print_timings:      sample time =     279.43 ms /   494 runs   (    0.57 ms per token,  1767.88 tokens per second)
llama_print_timings: prompt eval time =  347201.68 ms /   773 tokens (  449.16 ms per token,     2.23 tokens per second)
llama_print_timings:        eval time =  316783.32 ms /   493 runs   (  642.56 ms per token,     1.56 tokens per second)
llama_print_timings:       total time =  667448.33 ms /  1266 tokens



    
- Definition and symptoms:
  - Schizophrenia is a mental disorder characterized by positive symptoms (delusions, hallucinations, disorganized thinking, behavioral changes), negative symptoms (lack of emotion, social withdrawal), and cognitive impairments.
  - Prevalence: approximately 24 million people worldwide are affected (1 in 300).
  - Risk factors: male gender, earlier onset in males compared to females, genetic and environmental influences including childhood adversity, cannabis use during adolescence, infectious diseases, parental age, prenatal nutrition.
- Outcomes:
  - Significant distress and impairment in various areas of life for individuals diagnosed with schizophrenia.
  - Suicide rates are higher among people with schizophrenia compared to the general population.
  - Decreased life expectancy due to physical health problems is also common among people with schizophrenia.
- Treatment:
  - Antipsychotic medication, counseling, job training, social rehabilitation, an